<a href="https://colab.research.google.com/github/SamiraHabib/Fake_News/blob/main/projeto_fake_news_1_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Entendendo o dataset**

  Os dados utilizados neste notebook foram coletados através de web scrapping, e são referentes ao banco de dados do site [EU vs Desinfo](https://euvsdisinfo.eu/disinformation-cases/) que, por sua vez é baseado em análises do _East Stratcom Task Force_.

A _East Stratcom Task Force_ é uma organização para explicar melhor os valores, interesses e políticas da União Euripeia nos países parceiros da europa oriental. E trabalha em parceria com instituições e delegações da União Europeia nos países da Parceria Oriental, na Ásia Central e na Federação da Rússia.
O trabalho da task force consiste em conscientizar sobre o impacto negativo da desinformação que se origina em fontes pró Governo Russo e é disseminada na europa oriental e além.

  Este banco de dados possui uma série de avaliações de notícias que podem
fornecer uma representação parcial, distorcida ou falsa da realidade. Por se apresentarem como notícias, é possível inferir que estas tem a intenção de causar desinformação, desta forma, podem ser englobadas como _Fake News_ na definição do conceito utilizado neste trabalho.

Desta forma, o conteúdo deste banco de dados é formado, de cada entrada sendo avaliações de _Fake News_ feitas pelo EU vs Desinfo sobre acusações de desinformação em notícias que apoiam o governo da Russia (_Kremlin_).

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import nltk 
from nltk.corpus import stopwords 
import json
import re
from string import punctuation
from unicodedata import normalize
%pylab inline
import nltk
import gensim

Populating the interactive namespace from numpy and matplotlib


In [ ]:
!pip install stop-words

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('rslp')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [ ]:
files = pd.read_csv('/content/data.csv')

In [ ]:
files.head()

,Unnamed: 0,claims_id,claim_published,first_appearance,review_id,is_part_of,claim_reviewed,review_published,review_name,html_text,text,issue_id,keyword_id,keyword_name,country_id,country_name,appearances,has_parts,creative_work_id,type,url,author,claim,web_archive_url,abstract,in_language,start_time,end_time,organization_id,location,organization_name,image_id,image_type,image_content_url,language_id,language_name,language_code
0,0,/claims/100,2019-12-13T00:00:00+00:00,/news_articles/598,/claim_reviews/100,/issues/177,Ukraine has put itself in a situation when ext...,2019-12-16T00:00:00+00:00,Normandy summit results: the EU plays on Russi...,<p>This article misrepresents the actual Germa...,This article misrepresents the actual German a...,/issues/177,"['/keywords/61', '/keywords/76', '/keywords/77...","['Abandoned Ukraine', 'Angela Merkel', 'Emmanu...","['/countries/4', '/countries/9', '/countries/1...","['Russia', 'Ukraine', 'The West', 'EU', 'Germa...",/news_articles/598,"['/claims/75', '/claims/79', '/claims/80', '/c...",/news_articles/598,http://schema.org/NewsArticle,https://sputnik.by/columnists/20191213/1043442...,/organizations/262,/claims/100,https://web.archive.org/web/20191213153206/htt...,"На нынешней же встрече Зеленский (который, кст...",/languages/3,NaN,NaN,/organizations/262,/countries/55,sputnik.by // lifenews.ru,/image_objects/23,http://schema.org/ImageObject,https://api.veedoo.io/images/5e3150bc27830_2fd...,/languages/3,Russian,rus
1,1,/claims/1000,2019-09-26T00:00:00+00:00,/news_articles/1835,/claim_reviews/1000,/issues/166,Regardless who was behind the recent attack on...,2019-09-27T00:00:00+00:00,The US benefits from the attack on the Saudi o...,<p>No evidence is provided to support the clai...,No evidence is provided to support the claim. ...,/issues/166,"['/keywords/4', '/keywords/10', '/keywords/111']","['Conspiracy', 'Terrorism', 'Donald Trump']","['/countries/2', '/countries/3', '/countries/58']","['Iran', 'United States', 'Saudi Arabia']",/news_articles/1835,"['/claims/934', '/claims/941', '/claims/942', ...",/news_articles/1835,http://schema.org/NewsArticle,https://southfront.org/who-really-benefits-fro...,/organizations/205,/claims/1000,https://web.archive.org/web/20190927121413/htt...,NaN,/languages/7,NaN,NaN,/organizations/205,NaN,southfront.org,NaN,NaN,NaN,/languages/7,English,eng
2,2,/claims/1002,2019-09-23T00:00:00+00:00,/media_objects/1837,/claim_reviews/1002,/issues/166,"Pilsudski is a historical figure, who establis...",2019-09-27T00:00:00+00:00,The Polish Legions of Pilsudski organized the ...,<p>This message is a part of the Kremlin’s pol...,This message is a part of the Kremlin’s policy...,/issues/166,"['/keywords/36', '/keywords/40', '/keywords/42...","['Genocide', 'WWII', 'Historical revisionism',...","['/countries/9', '/countries/24']","['Ukraine', 'Poland']",/media_objects/1837,"['/claims/934', '/claims/941', '/claims/942', ...",/media_objects/1837,http://schema.org/MediaObject,https://youtube.com/watch?v=9wvNBUNML3Q&t=250,/organizations/122,/claims/1002,https://web.archive.org/web/20190927133752/htt...,"Пилсудский фигура историческая, создавшая перв...",/languages/3,250.0,262.0,/organizations/122,/countries/4,Rossia 24,/image_objects/10,http://schema.org/ImageObject,https://api.veedoo.io/images/5e313449bca43_Ros...,/languages/3,Russian,rus
3,3,/claims/1003,2019-09-27T00:00:00+00:00,/media_objects/1838,/claim_reviews/1003,/issues/166,Washington (and to a large degree Brussels) ar...,2019-09-27T00:00:00+00:00,The West might use the Eastern Europeans as ca...,"<p><a href=""https://euvsdisinfo.eu/why-authori...","Conspiracy theory, presented without evidence....",/issues/166,"['/keywords/20', '/keywords/107', '/keywords/2...","['Russophobia', 'Encircling Russia', 'Operatio...","['/countries/3', '/countries/11', '/countries/...","['United States', 'The West', 'Czech Republic'...",/media_objects/1838,"['/claims/934', '/claims/941', '/claims/942', ...",/media_objects/1838,http://schema.or

##**Dicionário de dados**

**Dicionário dos dados do dataset original:**

- 'Unnamed: 0': Número  único de identificação de cada linha de entrada, identico ao index.

- 'claims_id': Número único de identificação do alerta (Claim) de FakeNews.

- 'claim_published': Data e hora da publicação do alerta de FakeNews.

- 'first_appearance': A primeira ocorrência de um alerta de FakeNews pra uma notícia.

-  'review_id': Número único de identificação da avaliação da veracidade daquela notícia pelo EU vs Desinfo.

- 'is_part_of': Número único de identificação da _Issue_.

- 'claim_reviewed': Resumo da notícia original feita pelo EU vs Desinfo.

- 'review_published': Data e hora da públicação da avaliação da veracidade da notícia, ou seja, se ela é ou não fake news.

- 'review_name': Título da avaliação de Fake News.

- 'html_text': Conclusão da acusação de fake news feita pelo EU vs Desinfo ainda em formato html.

- 'text': Conclusão da acusação de fake news.

- 'issue_id': Número único de identificação da 'Issue'.

- 'keyword_id': Número único de identificação da palavra-chave.

- 'keyword_name': Nome da palavra-chave.

-'country_id': Número único de identificação dos países que são mensionados na notícia.

- 'country_name': Nome dos países que são mensionados na notícia.

- 'appearances': A primeira ocorrência de um alerta de FakeNews pra uma notícia.

- 'has_parts': Número único de identificação (id) de alertas de FakeNews feitas para aquela notícia.

- 'creative_work_id': Número único de identificação de um trabalho criativo, igual a coluna first_appearence.

- 'type': Tipo do conteúdo que foi avaliado pelo US vs Definfo, temos avaliações de [artigos](http://schema.org/NewsArticle) e [mídia](https://schema.org/MediaObject).

- 'url': Endereço web referente a cada notícia original.

- 'author': Número unico de identificação da instituição que produziu a notícia original.

- 'claim':  Número único de identificação do alerta de FakeNews.

- 'web_archive_url': Arquivo salvo (html) da página web da notícia original.

- 'abstract': Trecho aleatório da notícia original.

- 'in_language': Número único de identificação da lingua utilizada na reportagem original.

- 'start_time'
- 'end_time'

- 'organization_id': Número unico de identificação da instituição que produziu a notícia original

- 'location': Número unico de identificação do país que está localizado organização que produziu a notícia original.

- 'organization_name': Nome da instituição que produziu o conteúdo da notícia original.

- 'image_id': Número único de identificação da logo marca da organização da notícia original.

- 'image_type': Tipo do objeto de mídia classificado pelo schema, neste caso, objeto de imagem.

- 'image_content_url': Logo marca da organização da notícia original.

- 'language_id': Número único de identificação da lingua utilizada na reportagem original.

- 'language_name': Nome da lingua utilizada na reportagem.

- 'language_code': Representação do nome da língua pelas três primeiras letras.

**Descobrindo o que significa: Criando o dicionário**

- *Selecionamos colunas específicas em ingles (pra facilitar o entendimento do site original) e analisamos as informações das colunas*

In [ ]:
files[files['language_code'] == 'eng'].iloc[0]

Unnamed: 0                                                           1
claims_id                                                 /claims/1000
claim_published                              2019-09-26T00:00:00+00:00
first_appearance                                   /news_articles/1835
review_id                                          /claim_reviews/1000
is_part_of                                                 /issues/166
claim_reviewed       Regardless who was behind the recent attack on...
review_published                             2019-09-27T00:00:00+00:00
review_name          The US benefits from the attack on the Saudi o...
html_text            <p>No evidence is provided to support the clai...
text                 No evidence is provided to support the claim. ...
issue_id                                                   /issues/166
keyword_id            ['/keywords/4', '/keywords/10', '/keywords/111']
keyword_name               ['Conspiracy', 'Terrorism', 'Donald Trump']
countr

- *Acessamos a url da notícia original*

In [ ]:
files[files['language_code'] == 'eng']['url'].iloc[0]

'https://southfront.org/who-really-benefits-from-iranian-attack-on-saudi-arabia/'

- *Acessamos também a avaliação da FakeNews no site da EU vs Desinfo, pesquisando por data e palavra chave diretamente no site, chegamos no  [link](https://euvsdisinfo.eu/report/the-us-benefits-from-the-attack-on-the-saudi-oil-facility) referente a notícia.*

##**Dropando colunas inúteis e duplicadas**

Como podemos perceber no **dicionário dos dados do dataset original**, a coluna 'Unnamed: 0' não é necessária pois trata-se de uma coluna com os indexes do CSV na transformação deste arquivo pra um DataFrame. 
Existem também, colunas com informações repetidas e, para facilitar o mapeamento, destacamos as que se repetem por tópicos de colunas iguais:

- is_part_of -  inssue_id

- appearances - creative_work_ID - fist apparence

- claim - claim_ID

- in_language - language_id

- organization_id - author

- html_text - text

Diante deste problema, vamos eliminar colunas duplicadas e inúteis. Mas vamos também melhorar a identificação dos dados do nosso DataFrame renomeando todas as colunas com uma melhor escolha semântica.



In [ ]:
files.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7369 entries, 0 to 7368
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         7369 non-null   int64  
 1   claims_id          7369 non-null   object 
 2   claim_published    7369 non-null   object 
 3   first_appearance   7369 non-null   object 
 4   review_id          7369 non-null   object 
 5   is_part_of         7369 non-null   object 
 6   claim_reviewed     7366 non-null   object 
 7   review_published   7369 non-null   object 
 8   review_name        7369 non-null   object 
 9   html_text          7369 non-null   object 
 10  text               7369 non-null   object 
 11  issue_id           7369 non-null   object 
 12  keyword_id         7369 non-null   object 
 13  keyword_name       7369 non-null   object 
 14  country_id         7369 non-null   object 
 15  country_name       7369 non-null   object 
 16  appearances        7369 

In [ ]:
del_cols = ['is_part_of', 'appearances', 'creative_work_id', 'claim', 'in_language', 'Unnamed: 0', 'html_text', 'organization_id']
files.drop(del_cols, axis = 1, inplace = True)

In [ ]:
files.head(2)

,claims_id,claim_published,first_appearance,review_id,claim_reviewed,review_published,review_name,text,issue_id,keyword_id,keyword_name,country_id,country_name,has_parts,type,url,author,web_archive_url,abstract,start_time,end_time,location,organization_name,image_id,image_type,image_content_url,language_id,language_name,language_code
0,/claims/100,2019-12-13T00:00:00+00:00,/news_articles/598,/claim_reviews/100,Ukraine has put itself in a situation when ext...,2019-12-16T00:00:00+00:00,Normandy summit results: the EU plays on Russi...,This article misrepresents the actual German a...,/issues/177,"['/keywords/61', '/keywords/76', '/keywords/77...","['Abandoned Ukraine', 'Angela Merkel', 'Emmanu...","['/countries/4', '/countries/9', '/countries/1...","['Russia', 'Ukraine', 'The West', 'EU', 'Germa...","['/claims/75', '/claims/79', '/claims/80', '/c...",http://schema.org/NewsArticle,https://sputnik.by/columnists/20191213/1043442...,/organizations/262,https://web.archive.org/web/20191213153206/htt...,"На нынешней же встрече Зеленский (который, кст...",NaN,NaN,/countries/55,sputnik.by // lifenews.ru,/image_objects/23,http://schema.org/ImageObject,https://api.veedoo.io/images/5e3150bc27830_2fd...,/languages/3,Russian,rus
1,/claims/1000,2019-09-26T00:00:00+00:00,/news_articles/1835,/claim_reviews/1000,Regardless who was behind the recent attack on...,2019-09-27T00:00:00+00:00,The US benefits from the attack on the Saudi o...,No evidence is provided to support the claim. ...,/issues/166,"['/keywords/4', '/keywords/10', '/keywords/111']","['Conspiracy', 'Terrorism', 'Donald Trump']","['/countries/2', '/countries/3', '/countries/58']","['Iran', 'United States', 'Saudi Arabia']","['/claims/934', '/claims/941', '/claims/942', ...",http://schema.org/NewsArticle,https://southfront.org/who-really-benefits-fro...,/organizations/205,https://web.archive.org/web/20190927121413/htt...,NaN,NaN,NaN,NaN,southfront.org,NaN,NaN,NaN,/languages/7,English,eng


In [ ]:
files.columns

Index(['claims_id', 'claim_published', 'first_appearance', 'review_id',
       'claim_reviewed', 'review_published', 'review_name', 'text', 'issue_id',
       'keyword_id', 'keyword_name', 'country_id', 'country_name', 'has_parts',
       'type', 'url', 'author', 'web_archive_url', 'abstract', 'start_time',
       'end_time', 'location', 'organization_name', 'image_id', 'image_type',
       'image_content_url', 'language_id', 'language_name', 'language_code'],
      dtype='object')

##**Novo dicionário de dados**

**colunas renomeadas**

- 'claims_id': Número único de identificação do alerta (Claim) de FakeNews.

- 'claim_datetime': Data e hora da publicação do alerta de FakeNews.

- 'claim_first': A primeira ocorrência de um alerta de FakeNews pra uma notícia.

-  'review_id': Número único de identificação da avaliação da veracidade daquela notícia pelo EU vs Desinfo.

- 'review_resume': Resumo da notícia original feita pelo EU vs Desinfo.

 'review_datetime': Data e hora da públicação da avaliação da veracidade da notícia, ou seja, se ela é ou não fake news.

- 'review_title': Título da avaliação de Fake News.

- 'review_conclusion_html': Conclusão da acusação de fake news feita pelo EU vs Desinfo ainda em formato html.

- 'review_conclusion': Conclusão da acusação de fake news feita pelo EU vs Desinfo.

- 'issue_id': Número único de identificação da 'Issue'.

- 'news_keyword_id': Número único de identificação de palavras-chave que aparecem nas notícias originais.

- 'news_keyword_name': Nomes das palavras-chave que aparecem nas notícias originais.

-'news_countrys_id': Número único de identificação dos países que são mensionados nas notícias originais.

- 'news_countrys_name': Nome dos países que são mensionados nas notícias originais.

- 'claims_total': Número único de identificação (id) de alertas de FakeNews feitas para aquela notícia.

- 'news_type': Tipo do conteúdo que foi avaliado pelo US vs Definfo, temos avaliações de [artigos](http://schema.org/NewsArticle) e [mídia](https://schema.org/MediaObject).

- 'news_url': Endereço web referente a cada notícia original.

- 'news_organization_id': Número unico de identificação da instituição que produziu a notícia original.

- 'news_url_archive': Arquivo salvo (html) da página web da notícia original.

- 'news_part': Trecho aleatório da notícia original.

- 'review_start_time'

- 'review_end_time'

- 'news_organization_location': Número unico de identificação do país que está localizado organização que produziu a notícia original.

- 'news_organization_name': Nome da instituição que produziu o conteúdo da notícia original.

- 'news_logo_id': Número único de identificação da logo marca da organização da notícia original.

- 'news_logo_type': Tipo do objeto de mídia classificado pelo schema, neste caso, objeto de imagem.

- 'news_logo_url': Logo marca da organização da notícia original.

- 'news_language_id': Número único de identificação da lingua utilizada na reportagem original.

- 'news_language_name': Nome da lingua utilizada na reportagem.

- 'news_language_code': Representação do nome da língua pelas três primeiras letras.



In [ ]:
rename_columns = ['claims_id', 'claim_datetime', 'claim_first', 'review_id', 'review_resume', 
'review_datetime','review_title', 'review_conclusion','issue_id', 'news_keyword_id', 'news_keyword',
'news_countries_id', 'news_countries', 'claims_total', 'news_type', 'news_url', 'news_org_id', 'news_url_archive', 
'news_part','review_start_time', 'review_end_time', 'news_org_location','news_org',
'news_logo_id','news_logo_type', 'news_logo_url', 'news_language_id', 'news_language', 'news_language_code']

In [ ]:
files.columns = rename_columns

In [ ]:
files.head(2)

,claims_id,claim_datetime,claim_first,review_id,review_resume,review_datetime,review_title,review_conclusion,issue_id,news_keyword_id,news_keyword,news_countries_id,news_countries,claims_total,news_type,news_url,news_org_id,news_url_archive,news_part,review_start_time,review_end_time,news_org_location,news_org,news_logo_id,news_logo_type,news_logo_url,news_language_id,news_language,news_language_code
0,/claims/100,2019-12-13T00:00:00+00:00,/news_articles/598,/claim_reviews/100,Ukraine has put itself in a situation when ext...,2019-12-16T00:00:00+00:00,Normandy summit results: the EU plays on Russi...,This article misrepresents the actual German a...,/issues/177,"['/keywords/61', '/keywords/76', '/keywords/77...","['Abandoned Ukraine', 'Angela Merkel', 'Emmanu...","['/countries/4', '/countries/9', '/countries/1...","['Russia', 'Ukraine', 'The West', 'EU', 'Germa...","['/claims/75', '/claims/79', '/claims/80', '/c...",http://schema.org/NewsArticle,https://sputnik.by/columnists/20191213/1043442...,/organizations/262,https://web.archive.org/web/20191213153206/htt...,"На нынешней же встрече Зеленский (который, кст...",NaN,NaN,/countries/55,sputnik.by // lifenews.ru,/image_objects/23,http://schema.org/ImageObject,https://api.veedoo.io/images/5e3150bc27830_2fd...,/languages/3,Russian,rus
1,/claims/1000,2019-09-26T00:00:00+00:00,/news_articles/1835,/claim_reviews/1000,Regardless who was behind the recent attack on...,2019-09-27T00:00:00+00:00,The US benefits from the attack on the Saudi o...,No evidence is provided to support the claim. ...,/issues/166,"['/keywords/4', '/keywords/10', '/keywords/111']","['Conspiracy', 'Terrorism', 'Donald Trump']","['/countries/2', '/countries/3', '/countries/58']","['Iran', 'United States', 'Saudi Arabia']","['/claims/934', '/claims/941', '/claims/942', ...",http://schema.org/NewsArticle,https://southfront.org/who-really-benefits-fro...,/organizations/205,https://web.archive.org/web/20190927121413/htt...,NaN,NaN,NaN,NaN,southfront.org,NaN,NaN,NaN,/languages/7,English,eng


##**Transformando pseudo listas em listas reais**

*As colunas que possuem lista na verdade apenas estão representadas como listas, mas são na verdade strings. Para resolver este problema, vamos utilizar a função EVAL, que reconhece um padrão de estruturas em python e transforma a entrada neste padrão identificado*

In [ ]:
from itertools import chain

In [ ]:
notList_list = ['news_countries','news_keyword_id','news_keyword','news_countries_id','claims_total']

In [ ]:
files['news_countries']  = files['news_countries'].apply(lambda x: eval(x))
files['news_keyword_id'] = files['news_keyword_id'].apply(lambda x: eval(x))
files['news_keyword'] = files['news_keyword'].apply(lambda x: eval(x))
files['news_countries_id'] = files['news_countries_id'].apply(lambda x: eval(x))
files['claims_total'] = files['claims_total'].apply(lambda x: eval(x))

##**Limpando ruídos do texto**

**Removendo as linhas vazias de texto**

In [ ]:
files[files['review_resume'].isnull() == True].index

Int64Index([3934, 3935, 3939], dtype='int64')

In [ ]:
files.dropna(subset = ['review_resume'], inplace = True) 

In [ ]:
files.reset_index(inplace= True)

In [ ]:
files.pop('index')

0          0
1          1
2          2
3          3
4          4
        ... 
7361    7364
7362    7365
7363    7366
7364    7367
7365    7368
Name: index, Length: 7366, dtype: int64

In [ ]:
files[files['review_resume'].isnull() == True]

,claims_id,claim_datetime,claim_first,review_id,review_resume,review_datetime,review_title,review_conclusion,issue_id,news_keyword_id,news_keyword,news_countries_id,news_countries,claims_total,news_type,news_url,news_org_id,news_url_archive,news_part,review_start_time,review_end_time,news_org_location,news_org,news_logo_id,news_logo_type,news_logo_url,news_language_id,news_language,news_language_code


**Removendo pontuação, caracteres especiais e transformando em minúsculo**

In [ ]:
import json
import re
from string import punctuation
from unicodedata import normalize

In [ ]:
def pre_process (column):

  df_preprocess = pd.DataFrame()

  #Retirando acentuação e transformando em minúsculo
  normalized_list = []

  for item in column:
    norm = normalize('NFKD', item).encode('ASCII', 'ignore').decode('ASCII')
    normalized_list.append(norm.lower())

  df_preprocess['normalized'] = normalized_list

  #Retirando expressões regulares
  cleam_words = []
  text_garbage = ['\r', '\n', "he\xa0", "»", "«", "'", '\"', ".", ",", ")", "(", "http"]

  for word in df_preprocess['normalized']:
    new_word = re.sub(r'[0-9]|,|\.|/|$|\(|\)|-|\+|:|•',"", word)
    if new_word != "":
      cleam_words.append(new_word)

  df_preprocess['cleam_words'] = cleam_words
  for i in text_garbage:
    df_preprocess['cleam_words'] = df_preprocess['cleam_words'].replace(i, '')


  return df_preprocess['cleam_words']

In [ ]:
files['review_resume'] = pre_process(files['review_resume'])
files['review_title'] = pre_process(files['review_title'])
files['review_conclusion'] = pre_process(files['review_conclusion'])

##**Recuperando informação**

In [ ]:
files.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7366 entries, 0 to 7365
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   claims_id           7366 non-null   object 
 1   claim_datetime      7366 non-null   object 
 2   claim_first         7366 non-null   object 
 3   review_id           7366 non-null   object 
 4   review_resume       7366 non-null   object 
 5   review_datetime     7366 non-null   object 
 6   review_title        7366 non-null   object 
 7   review_conclusion   7366 non-null   object 
 8   issue_id            7366 non-null   object 
 9   news_keyword_id     7366 non-null   object 
 10  news_keyword        7366 non-null   object 
 11  news_countries_id   7366 non-null   object 
 12  news_countries      7366 non-null   object 
 13  claims_total        7366 non-null   object 
 14  news_type           7366 non-null   object 
 15  news_url            7366 non-null   object 
 16  news_o

*Percorremos cada item das duas colunas que possuem informações do ID de cada país utilizado nesse dataset e o nome dos países referentes a estes IDs, respectivamente, e transformamos em duas grandes listas diferentes mantendo a ordem em que os ítens aparecem, inclusive as repetições.*

In [ ]:
countries_id = list(chain(*files['news_countries_id']))
countries_name = list(chain(*files['news_countries']))

*Agora vamos unir os itens das duas listas de acordo com as suas respectivas posições na ordem das listas. Excluir os que se repetem e transformar em um dicionário*

In [ ]:
countries_dict = dict(set(zip(countries_id, countries_name)))

In [ ]:
countries_dict

{'/countries/1': 'Qatar',
 '/countries/10': 'West',
 '/countries/100': 'Switzerland',
 '/countries/101': 'Luxembourg',
 '/countries/102': 'FYROM',
 '/countries/103': 'Dominican Republic',
 '/countries/104': 'Mongolia',
 '/countries/105': 'India',
 '/countries/106': 'South Korea',
 '/countries/107': 'Ireland',
 '/countries/108': 'Cyprus',
 '/countries/109': 'CEE',
 '/countries/11': 'The West',
 '/countries/110': 'SEE',
 '/countries/111': 'Chechnya',
 '/countries/112': 'Sudan',
 '/countries/113': 'Islam',
 '/countries/115': 'Yemen',
 '/countries/116': 'David Icke',
 '/countries/117': 'Al-Qaeda',
 '/countries/118': 'Daesh',
 '/countries/119': 'Mali',
 '/countries/12': 'Montenegro',
 '/countries/13': 'Europe',
 '/countries/14': 'EU',
 '/countries/15': 'The Netherlands',
 '/countries/17': 'Syria',
 '/countries/18': 'Turkey',
 '/countries/19': 'Italy',
 '/countries/2': 'Iran',
 '/countries/20': 'Estonia',
 '/countries/21': 'North Macedonia',
 '/countries/22': 'Serbia',
 '/countries/23': 'Bos

*Com a estrutura de dicionário criada, vamos agora substituir na coluna 'news_organization_location' os IDs dos países pelos nomes correspondentes.*

In [ ]:
len(files[files['news_org_location'].isnull() == True])

5593

In [ ]:
files.replace({'news_org_location' : countries_dict}, inplace = True)

In [ ]:
files['news_org_location']

0       Belarus
1           NaN
2        Russia
3           NaN
4           NaN
         ...   
7361        NaN
7362        NaN
7363        NaN
7364        NaN
7365        NaN
Name: news_org_location, Length: 7366, dtype: object

##**Verificando claims duplicadas**

In [ ]:
len(files[files['claims_id'].duplicated()])

1128

In [ ]:
len(files[files['claims_id'].duplicated()]['claims_id'].unique())

591

In [ ]:
files[files['claims_id'].duplicated()]['claims_id'].unique()

array(['/claims/101', '/claims/1022', '/claims/104', '/claims/1049',
       '/claims/1053', '/claims/1055', '/claims/1060', '/claims/1061',
       '/claims/1083', '/claims/1118', '/claims/112', '/claims/1158',
       '/claims/117', '/claims/1171', '/claims/1210', '/claims/1235',
       '/claims/1236', '/claims/1254', '/claims/129', '/claims/1294',
       '/claims/131', '/claims/1317', '/claims/1327', '/claims/134',
       '/claims/1344', '/claims/1365', '/claims/1407', '/claims/1413',
       '/claims/1426', '/claims/1453', '/claims/146', '/claims/1464',
       '/claims/147', '/claims/148', '/claims/151', '/claims/1516',
       '/claims/1521', '/claims/153', '/claims/1532', '/claims/1557',
       '/claims/1558', '/claims/158', '/claims/1584', '/claims/1596',
       '/claims/160', '/claims/161', '/claims/162', '/claims/1682',
       '/claims/1684', '/claims/1685', '/claims/1701', '/claims/1705',
       '/claims/1719', '/claims/173', '/claims/1730', '/claims/1732',
       '/claims/1749', 

In [ ]:
files[files['claims_id'] == '/claims/994']

,claims_id,claim_datetime,claim_first,review_id,review_resume,review_datetime,review_title,review_conclusion,issue_id,news_keyword_id,news_keyword,news_countries_id,news_countries,claims_total,news_type,news_url,news_org_id,news_url_archive,news_part,review_start_time,review_end_time,news_org_location,news_org,news_logo_id,news_logo_type,news_logo_url,news_language_id,news_language,news_language_code
7354,/claims/994,2019-09-27T00:00:00+00:00,/news_articles/1823,/claim_reviews/994,in the village of starychi a mans body was fou...,2019-09-28T00:00:00+00:00,ukrainian soldier shot by polish soldier near ...,no evidence is given to back up these claims t...,/issues/166,"[/keywords/105, /keywords/150]","[Military, Military exercise]","[/countries/9, /countries/24]","[Ukraine, Poland]","[/claims/934, /claims/941, /claims/942, /claim...",http://schema.org/NewsArticle,https://vesti.ua/strana/351746-pod-lvovom-pols...,/organizations/458,https://web.archive.org/web/20190928204050/htt...,В селе Старычи было найдено тело мужчины с огн...,NaN,NaN,NaN,Vesti.ua,NaN,NaN,NaN,/languages/14,Polish,pol
7355,/claims/994,2019-09-27T00:00:00+00:00,/news_articles/1823,/claim_reviews/994,in the village of starychi a mans body was fou...,2019-09-28T00:00:00+00:00,ukrainian soldier shot by polish soldier near ...,no evidence is given to back up these claims t...,/issues/166,"[/keywords/105, /keywords/150]","[Military, Military exercise]","[/countries/9, /countries/24]","[Ukraine, Poland]","[/claims/934, /claims/941, /claims/942, /claim...",http://schema.org/NewsArticle,https://life.ru/t/%D0%BD%D0%BE%D0%B2%D0%BE%D1%...,/organizations/264,https://web.archive.org/web/20190928204125/htt...,NaN,NaN,NaN,NaN,life.ru,NaN,NaN,NaN,/languages/3,Russian,rus
7356,/claims/994,2019-09-27T00:00:00+00:00,/news_articles/1823,/claim_reviews/994,in the village of starychi a mans body was fou...,2019-09-28T00:00:00+00:00,ukrainian soldier shot by polish soldier near ...,no evidence is given to back up these claims t...,/issues/166,"[/keywords/105, /keywords/150]","[Military, Military exercise]","[/countries/9, /countries/24]","[Ukraine, Poland]","[/claims/934, /claims/941, /claims/942, /claim...",http://schema.org/NewsArticle,https://lenta.ru/news/2019/09/25/polskiebratya/,/organizations/226,https://web.archive.org/web/20190928204208/htt...,NaN,NaN,NaN,NaN,lenta.ru,NaN,NaN,NaN,/languages/14,Polish,pol
7357,/claims/994,2019-09-27T00:00:00+00:00,/news_articles/1823,/claim_reviews/994,in the village of starychi a mans body was fou...,2019-09-28T00:00:00+00:00,ukrainian soldier shot by polish soldier near ...,no evidence is given to back up these claims t...,/issues/166,"[/keywords/105, /keywords/150]","[Military, Military exercise]","[/countries/9, /countries/24]","[Ukraine, Poland]","[/claims/934, /claims/941, /claims/942, /claim...",http://schema.org/NewsArticle,https://www.mk.ru/politics/2019/09/25/pod-lvov...,/organizations/34,https://web.archive.org/web/20190928204245/htt...,NaN,NaN,NaN,NaN,mk.ru,/image_objects/31,http://schema.org/ImageObject,https://api.veedoo.io/images/5e31565e0a12e_DET...,/languages/14,Polish,pol
7358,/claims/994,2019-09-27T00:00:00+00:00,/news_articles/1823,/claim_reviews/994,in the village of starychi a mans body was fou...,2019-09-28T00:00:00+00:00,ukrainian soldier shot by polish soldier near ...,no evidence is given to back up these claims t...,/issues/166,"[/keywords/105, /keywords/150]","[Military, Military exercise]","[/countries/9, /countries/24]","[Ukraine, Poland]","[/claims/934, /claims/941, /claims/942, /claim...",http://schema.org/NewsArticle,https://lviv.politeka.net/lviv/1171159-polskog...,/organizations/459,https://web.archive.org/web/20190928204346/htt...,NaN,NaN,NaN,NaN,Politeka.net,NaN,NaN,NaN,/languages/14,Polish,pol
7359,/claims/994,2019-09-27T00:00:00+00:00,/news_articles/1823,/claim_reviews/994,in the village of starychi a mans body was fou...,2019-09-28T00:00:00+00:00,ukrainian soldier shot by polish soldier near ...,no eviden

In [ ]:
files[files['claims_id'] == '/claims/994']['review_conclusion'][7354]

'no evidence is given to back up these claims the press service of the police of the lviv region said that "information about this event is not true and the communications department of the police of the lviv region did not prepare or distribute this material" the publication used fake screenshots allegedly from the official site of the lviv police\nalso later the newspaper ratusha which was the first one to share this information made a statement that their editorial office did not post this news item on its website alleging that their website was hacked\nfurther debunking by keyboard warriors wojownicy klawiatury and konkret\n'

In [ ]:
files[files['claims_id'] == '/claims/71']['review_conclusion'][5809]

'recurring prokremlin narrative seeking to absolve the assad regime of responsibility for chemical attacks perpetrated in the course of the syrian civil war as well as to undermine the credibility and independence of the opcw they claim that the douma chemical attacks were staged by western actors in order to justify military action against the assad regime\nthe internal opcw email message published by wikileaks sent on  june  by an unidentified member of opcws factfinding mission ffm does not accuse the opcw management of deliberately manipulating and suppressing evidence gathered by the ffm in order to blame the assad government for the douma attack and to justify western military intervention against syria\nfor example the management\'s draft said that chlorine or another reactive chlorinecontaining chemical was "likely" to have been released from two cylinders found at the scene the email argues instead that while the cylinders might have been the sources of the suspected chemical 

*As revisões de fake news ou claims repetidas, são denuncias para notícias totalmente iguais encontradas em sites diferentes. Sendo suas únicas as urls e as organizações que publicam as notícias*

*Por enquanto, vamos salvar essas claims duplicadas em um dataframe a parte para uma análise posterior e dropar do dataframe principal que será utilizado nos notebooks seguintes.*

In [ ]:
df_duplicates = pd.DataFrame()
df_duplicates = files[files['claims_id'].duplicated()]

In [ ]:
files.drop_duplicates(subset = ['claims_id'], keep = 'first', inplace = True)

In [ ]:
files.reset_index()

,index,claims_id,claim_datetime,claim_first,review_id,review_resume,review_datetime,review_title,review_conclusion,issue_id,news_keyword_id,news_keyword,news_countries_id,news_countries,claims_total,news_type,news_url,news_org_id,news_url_archive,news_part,review_start_time,review_end_time,news_org_location,news_org,news_logo_id,news_logo_type,news_logo_url,news_language_id,news_language,news_language_code
0,0,/claims/100,2019-12-13T00:00:00+00:00,/news_articles/598,/claim_reviews/100,ukraine has put itself in a situation when ext...,2019-12-16T00:00:00+00:00,normandy summit results the eu plays on russia...,this article misrepresents the actual german a...,/issues/177,"[/keywords/61, /keywords/76, /keywords/77, /ke...","[Abandoned Ukraine, Angela Merkel, Emmanuel Ma...","[/countries/4, /countries/9, /countries/11, /c...","[Russia, Ukraine, The West, EU, Germany, France]","[/claims/75, /claims/79, /claims/80, /claims/8...",http://schema.org/NewsArticle,https://sputnik.by/columnists/20191213/1043442...,/organizations/262,https://web.archive.org/web/20191213153206/htt...,"На нынешней же встрече Зеленский (который, кст...",NaN,NaN,Belarus,sputnik.by // lifenews.ru,/image_objects/23,http://schema.org/ImageObject,https://api.veedoo.io/images/5e3150bc27830_2fd...,/languages/3,Russian,rus
1,1,/claims/1000,2019-09-26T00:00:00+00:00,/news_articles/1835,/claim_reviews/1000,regardless who was behind the recent attack on...,2019-09-27T00:00:00+00:00,the us benefits from the attack on the saudi o...,no evidence is provided to support the claim a...,/issues/166,"[/keywords/4, /keywords/10, /keywords/111]","[Conspiracy, Terrorism, Donald Trump]","[/countries/2, /countries/3, /countries/58]","[Iran, United States, Saudi Arabia]","[/claims/934, /claims/941, /claims/942, /claim...",http://schema.org/NewsArticle,https://southfront.org/who-really-benefits-fro...,/organizations/205,https://web.archive.org/web/20190927121413/htt...,NaN,NaN,NaN,NaN,southfront.org,NaN,NaN,NaN,/languages/7,English,eng
2,2,/claims/1002,2019-09-23T00:00:00+00:00,/media_objects/1837,/claim_reviews/1002,pilsudski is a historical figure who establish...,2019-09-27T00:00:00+00:00,the polish legions of pilsudski organized the ...,this message is a part of the kremlins policy ...,/issues/166,"[/keywords/36, /keywords/40, /keywords/42, /ke...","[Genocide, WWII, Historical revisionism, WWI, ...","[/countries/9, /countries/24]","[Ukraine, Poland]","[/claims/934, /claims/941, /claims/942, /claim...",http://schema.org/MediaObject,https://youtube.com/watch?v=9wvNBUNML3Q&t=250,/organizations/122,https://web.archive.org/web/20190927133752/htt...,"Пилсудский фигура историческая, создавшая перв...",250.0,262.0,Russia,Rossia 24,/image_objects/10,http://schema.org/ImageObject,https://api.veedoo.io/images/5e313449bca43_Ros...,/languages/3,Russian,rus
3,3,/claims/1003,2019-09-27T00:00:00+00:00,/media_objects/1838,/claim_reviews/1003,washington and to a large degree brussels are ...,2019-09-27T00:00:00+00:00,the west might use the eastern europeans as ca...,conspiracy theory presented without evidence t...,/issues/166,"[/keywords/20, /keywords/107, /keywords/299]","[Russophobia, Encircling Russia, Operation Bar...","[/countries/3, /countries/11, /countries/64, /...","[United States, The West, Czech Republic, East...","[/claims/934, /claims/941, /claims/942, /claim...",http://schema.org/MediaObject,https://www.fort-russ.com/2019/09/debunking-li...,/organizations/461,NaN,NaN,NaN,NaN,NaN,fort-russ.com,NaN,NaN,NaN,/languages/7,English,eng
4,4,/claims/1004,2019-09-25T00:00:00+00:00,/news_articles/1839,/claim_reviews/1004,the beneficiary of the resolution of the europ...,2019-09-27T00:00:00+00:00,the resolution of the european parliament reli...,this message is part of the kremlins policy of...,/issues/166,"[/keywords/30, /keywords/40, /keywords/41, /ke...","[European Parliament, WWII, Adolf Hitler, Worl...","[/countries/4, /countries/30, /countries/66]","[Russia, Germany, Eastern Europe]","[/cla

##**Separando os DataFrames**

*Como cada entrada deste data frame é referente a revisão de um alerta de fake news de uma notícia, apenas o index para identificar cada revisão seria o suficiente, visto que os demais IDs já possuem sua coluna correspondente na tabela.
Diante disto, vamos separar os IDs em um DF separado, caso sejam necessários em consultas futuras e dropar do dataframe total.*

In [ ]:
id_list = []
for column in files.columns:
    if 'id' in column:
      id_list.append(column)
df_id = files[id_list]
df_total = files.drop(columns = id_list)

In [ ]:
print(f"Colunas do df total:{df_total.columns} \n\n Colunas do df id: {df_id.columns}")

Colunas do df total:Index(['claim_datetime', 'claim_first', 'review_resume', 'review_datetime',
       'review_title', 'review_conclusion', 'news_keyword', 'news_countries',
       'claims_total', 'news_type', 'news_url', 'news_url_archive',
       'news_part', 'review_start_time', 'review_end_time',
       'news_org_location', 'news_org', 'news_logo_type', 'news_logo_url',
       'news_language', 'news_language_code'],
      dtype='object') 

 Colunas do df id: Index(['claims_id', 'review_id', 'issue_id', 'news_keyword_id',
       'news_countries_id', 'news_org_id', 'news_logo_id', 'news_language_id'],
      dtype='object')


In [ ]:
df_total.head(2)

,claim_datetime,claim_first,review_resume,review_datetime,review_title,review_conclusion,news_keyword,news_countries,claims_total,news_type,news_url,news_url_archive,news_part,review_start_time,review_end_time,news_org_location,news_org,news_logo_type,news_logo_url,news_language,news_language_code
0,2019-12-13T00:00:00+00:00,/news_articles/598,ukraine has put itself in a situation when ext...,2019-12-16T00:00:00+00:00,normandy summit results the eu plays on russia...,this article misrepresents the actual german a...,"[Abandoned Ukraine, Angela Merkel, Emmanuel Ma...","[Russia, Ukraine, The West, EU, Germany, France]","[/claims/75, /claims/79, /claims/80, /claims/8...",http://schema.org/NewsArticle,https://sputnik.by/columnists/20191213/1043442...,https://web.archive.org/web/20191213153206/htt...,"На нынешней же встрече Зеленский (который, кст...",NaN,NaN,Belarus,sputnik.by // lifenews.ru,http://schema.org/ImageObject,https://api.veedoo.io/images/5e3150bc27830_2fd...,Russian,rus
1,2019-09-26T00:00:00+00:00,/news_articles/1835,regardless who was behind the recent attack on...,2019-09-27T00:00:00+00:00,the us benefits from the attack on the saudi o...,no evidence is provided to support the claim a...,"[Conspiracy, Terrorism, Donald Trump]","[Iran, United States, Saudi Arabia]","[/claims/934, /claims/941, /claims/942, /claim...",http://schema.org/NewsArticle,https://southfront.org/who-really-benefits-fro...,https://web.archive.org/web/20190927121413/htt...,NaN,NaN,NaN,NaN,southfront.org,NaN,NaN,English,eng


In [ ]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6238 entries, 0 to 7365
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   claim_datetime      6238 non-null   object 
 1   claim_first         6238 non-null   object 
 2   review_resume       6238 non-null   object 
 3   review_datetime     6238 non-null   object 
 4   review_title        6238 non-null   object 
 5   review_conclusion   6238 non-null   object 
 6   news_keyword        6238 non-null   object 
 7   news_countries      6238 non-null   object 
 8   claims_total        6238 non-null   object 
 9   news_type           6238 non-null   object 
 10  news_url            6238 non-null   object 
 11  news_url_archive    2701 non-null   object 
 12  news_part           790 non-null    object 
 13  review_start_time   356 non-null    float64
 14  review_end_time     356 non-null    float64
 15  news_org_location   1606 non-null   object 
 16  news_o

##**Criando coluna de texto corrido**

In [ ]:
df_total['review_total'] = df_total['review_title'] + df_total['review_resume'] + df_total['review_conclusion']

In [ ]:
def remove_stopwords(df, col):
  import re
  import sys
  from nltk.corpus import stopwords 
  from nltk.tokenize import word_tokenize

  df_total = pd.DataFrame()
  
  words_compr = []
  for idx, item in enumerate(df[col]):
    words_compr.append(item)
  words2 = "".join(map(str, words_compr))

  userstring = words2.split(" ")
  stop_words = set(stopwords.words('english')) 
  another_list_word = []
  for x in userstring:
    x = x.lower()
    if x not in stop_words:           
      another_list_word.append(x)
  
  df_total['stop_words'] = another_list_word

  return df_total['stop_words']

In [ ]:
remove_stopwords(df_total, 'review_total')

0              normandy
1                summit
2               results
3                    eu
4                 plays
              ...      
512519       supporting
512520           behind
512521        terrorist
512522    organisations
512523           here\n
Name: stop_words, Length: 512524, dtype: object

##**Salvando o json**

In [ ]:
df_total.to_json('fakeNews_FINAL.json')